In [ ]:
%matplotlib inline

import sys 
import os 
import numpy as np

nb_dir = os.getcwd()
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from plotlib.loaders import *
from plotlib.plotters import *

from phdconf import config 
from phdconf.config import *

In [ ]:
queries = load_queries(config.AUS_TOPIC_PATH)
tt_folds = read_folds(AUS_FOLDS)

In [ ]:
index_names = ['case-topics']
qrel_paths = [config.AUS_QREL_PATH] * len(index_names)
rel_levels = [config.AUS_REL_LEVEL] * len(index_names)
display_names = ['base', 'sec-count', 'leg-count', 'date', 'hierarchy']

topics = 'case-topics'

to = 99

In [ ]:
mu = 1050
base_df = load_1d_dfs(['filtered-phrasestop'], qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, mu, mu, 1)[0][0]
sdm_base_df = load_1d_dfs(['filtered-phrasestop'], qrel_paths, os.path.join(BASE_DIR, 'sdm_rerank'), 'case-topics-{0}-sdm_rerank-dir-mu-{1:.2f}-weights-0.85-0.00-0.15-window-2.run', rel_levels, mu, mu, 1)[0][0]
base_qry = load_1d_dfs(['filtered-phrasestop'], qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, mu, mu, 1, per_query=True)[0][0]

In [ ]:
# class Linker:
#     def __init__(self, path: str, id_path: str):
#         self._lookup = {}
#         with open(id_path) as f:
#             for line in f:
#                 parts = line.split()
#                 self._lookup[parts[0].upper()] = int(parts[1])

#         self._lookup_ind = ['']*len(self._lookup)
#         for k, v in self._lookup.items():
#             self._lookup_ind[v] = k
        
#         self._inlinks = [None]*len(self._lookup)
#         self._outlinks = [None]*len(self._lookup)

#         with open(path) as f:
#             for line in f:
#                 parts = list(map(int, line.split()))
                
#                 if self._inlinks[parts[1]] == None:
#                     self._inlinks[parts[1]] = []
#                 self._inlinks[parts[1]].append((parts[0], parts[2]))

#                 if self._outlinks[parts[0]] == None:
#                     self._outlinks[parts[0]] = []
#                 self._outlinks[parts[0]].append((parts[1], parts[2]))
                
#     def ids(self):
#         return self._lookup_ind
    
#     def out_links(self): 
#         return self._outlinks
    
#     def in_links(self): 
#         return self._inlinks
                
#     def get_links_for_id(_id: str, vals, count:bool = False):
#         ind = self._lookup.get(_id.toupper(), None)
#         if ind != None:
#             if count:
#                 _sum = 0
#                 for i in vals[ind]:
#                     _sum += i[1]
#                 return _sum
#             return len(vals[ind])
#         return 0

# linker = Linker(os.path.join(os.environ["HOME"], "go/src/cit-extract/links.txt"), os.path.join(os.environ["HOME"], "go/src/cit-extract/id-lookup.txt"))

In [ ]:
# # get list of documents and relevance list 
# def get_qrel_rel_docs(path: str): 
#     out = set()
#     with open(path) as f:
#         for line in f:
#             parts = line.strip().split()
#             if int(parts[3]) > 0:
#                 out.add(parts[2])
                
#     return out

# def get_bin(v, bins): 
#     for i in range(1, len(bins)): 
# #         print(v, bins)
#         if v < bins[i]: 
#             return i-1
#     return 0 

# def get_rel_vals(qrels, lookup, bins):
#     vals = [0.0] * len(qrels)
#     for i, q in enumerate(qrels):
#         vals[i] = lookup.get(q, True)
    
#     bin_cnt = [0] * (len(bins)-1)
#     for i, v in enumerate(vals): 
#         bin_cnt[get_bin(v, bins)] +=1
    
#     return bin_cnt
    
# def get_buckets(data):
#     print(data[:10])
#     df = pd.DataFrame(data)
#     bucketed, bins = pd.cut(df[0], 100, retbins=True)
#     return bucketed, bins

In [ ]:
# buckets = 54

# rel_docs = get_qrel_rel_docs(config.AUS_QREL_PATH)

# in_links = linker.in_links()
# out_links = linker.out_links()

# inlink = [len(x) if x is not None else 0 for x in in_links]
# outlink = [len(x) if x is not None else 0 for x in out_links]

# sum_in = []
# for x in in_links: 
#     if x is not None:
#         sum_in.append(sum([y[1] for y in x]))
#     else:
#         sum_in.append(0)
# sum_out = []
# for x in out_links:
#     if x is not None:
#         sum_out.append(sum([y[1] for y in x]))
#     else:
#         sum_out.append(0)

# rel = [True if x in rel_docs else False for x in linker.ids()]

# labels = [x for x in range(1, buckets+1)]

# len_df = pd.DataFrame({'id': [x.upper() for x in doc_lens.keys()], 'lens': list(doc_lens.values())}).set_index(['id'])

In [ ]:
inter = Interpolater(os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior', 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00.run'), normalize=True)

dfs = []
for d in display_names[1:]: 
    interped_dfs = []
    for _lambda in np.arange(0, 1.0, 0.01):
        inter.interpolate(os.path.join(BASE_DIR, 'case-topics-{0}.run'.format(d)), _lambda, 'tmp.run')
        interped_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'])[0])
    dfs.append(interped_dfs)

In [ ]:
to = 15
link_text_fig = plot_tune_1d_comp(['base', 'sections', 'legislation'], RERANK_METRICS, 
                    [[base_df for x in range(to+1)]] + [x[:to+1] for x in dfs[:2]], 0.00, (to)/100, 0.01, legend_x=0.965, ylims=RERANK_YLIMS, styles=['--'])
# link_text_fig.savefig('figures/ausnl-legref-interp.pdf')

In [ ]:
to = 99
link_text_fig = plot_tune_1d_comp(['base', 'date'], RERANK_METRICS, 
                    [[base_df for x in range(to+1)]] + [x[:to+1] for x in dfs[2:3]], 0.00, (to)/100, 0.01, legend_x=0.965, ylims=RERANK_YLIMS, styles=['--'])
# link_text_fig.savefig('figures/ausnl-date-interp.pdf')

In [ ]:
to = 40
link_text_fig = plot_tune_1d_comp(['base', 'hierarchy'], RERANK_METRICS, 
                    [[base_df for x in range(to+1)]] + [x[:to+1] for x in dfs[3:4]], 0.00, (to)/100, 0.01, legend_x=0.965, ylims=RERANK_YLIMS, styles=['--'])
# link_text_fig.savefig('figures/ausnl-hierarchy-interp.pdf')

In [ ]:
inter = Interpolater(os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior', 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00.run'), normalize=True)

q_dfs = []
for d in display_names[1:]: 
    interped_dfs = []
    for _lambda in np.arange(0, 1.0, 0.01):
        inter.interpolate(os.path.join(BASE_DIR, 'case-topics-{0}.run'.format(d)), _lambda, 'tmp.run')
        interped_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'], per_query=True)[0])
    q_dfs.append(interped_dfs)

In [ ]:
df = pd.DataFrame(columns=RERANK_METRICS)

for ab, runs in zip(['sec', 'leg', 'date', 'hierarchy'], q_dfs):
    cross = cross_validation(runs, tt_folds, RERANK_METRICS, base_qry)
    df.loc[ab] = cross[0]

In [ ]:
df.loc['$R$'] = base_df.round(4)

date_df = df.reindex(['$R$', 'date'])
# write_table('tables/ausnl-date', bold_max(date_df).rename(columns=RERANK_METRICS).drop('Unjudged@20',axis='columns').to_latex(escape=False))

In [ ]:
leg_df = df.reindex(['$R$', 'sec', 'leg'])
# write_table('tables/ausnl-leg', bold_max(leg_df).rename(columns=RERANK_METRICS).drop('Unjudged@20',axis='columns').to_latex(escape=False))

In [ ]:
h_df = df.reindex(['$R$', 'hierarchy'])
# write_table('tables/ausnl-hierarchy', bold_max(h_df).rename(columns=RERANK_METRICS).drop('Unjudged@20',axis='columns').to_latex(escape=False))

In [ ]:
link_max = select_1d_max_with_interp(display_names[1:], dfs, 0.0, 0.01, '$\lambda$', inter, base_qry, base_df, 1050, os.path.join(BASE_DIR, 'case-topics-{0}.run'), config.AUS_QREL_PATH, config.AUS_REL_LEVEL, metrics=RERANK_METRICS).T

In [ ]:
# print(link_max.drop(['Unjudged@20'], axis='columns').to_latex(escape=False))
link_max

In [ ]:
# max for err@20
om = copy.copy(config.METRIC_NAMES)
del om['recall_100']
del om['unjudged@20']

for r in display_names[1:]:
    interps = link_max.loc[r, '$\lambda$'].values
    cols = om.keys()
    a = pd.DataFrame()
    for i, c in zip(interps, cols):
        inter.interpolate(os.path.join(BASE_DIR, 'case-topics-{0}.run'.format(r)), float(i), 'tmp.run')
        a[c] = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'], per_query=True)[0][c]

    qry_comp_df = a-base_qry
    qry_comp_fig = qry_comp_df[om.keys()].rename(RERANK_METRICS, axis='columns').plot.box(fontsize=15, boxprops=dict(linestyle='-', linewidth=2), medianprops=dict(linestyle='-', linewidth=2), color=dict(boxes='black', whiskers='black', medians='b', caps='r'), figsize=(16, 4)).axhline(y=0, xmin=0.0, xmax=1.0, linestyle='--', linewidth=1.0, color='grey')
    # qry_comp_fig.get_figure().savefig('figures/ausnl-{0}-qry-comp.pdf'.format(r))

## Combining multiple features

In [ ]:
max_runs = [os.path.join(BASE_DIR, 'grid-search', '{0}-combine-max.run'.format(x)) for x in ['RR', 'ERR', 'R@20', 'NDCG', 'RBP']]
print(max_runs)
max_run_res = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', max_runs)

In [ ]:
pd.DataFrame(max_run_res, columns=om)

In [ ]:
max_runs = [os.path.join(BASE_DIR, 'grid-search', '4feat-{0}-combine-max.run'.format(x)) for x in ['RR', 'ERR', 'R@20', 'NDCG', 'RBP']]
print(max_runs)
max_run_res = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', max_runs)

In [ ]:
pd.DataFrame(max_run_res, columns=om)

In [ ]:
# def create_folds(queries, k: int):
#     l = len(queries)
#     q_set = set(queries)
#     inds = [x for x in range(l)]
#     np.random.shuffle(inds)
#     queries = np.array(queries)
#     queries = queries[inds]
    
#     avg = l / float(k)
#     parts = []
#     last = 0.0
    
#     test = []

#     while last < l:
#         test.append(list(queries[int(last):int(last + avg)]))
#         last += avg
     
#     for i in range(len(parts)):
#         train.append()
    
#     train = []
#     for i in range(len(test)):
#         train.append(list(q_set.difference(test[i])))
    
#     return train, test
    
# train, test = create_folds(list(queries.keys()), 5)

In [ ]:
# # write folds to file 
# with open('ausnl-folds.txt', 'w') as f:
#     for tr, te in zip(train, test):
#         f.write('{0} {1}\n'.format(tr, te))

In [ ]:
tt_folds = read_folds(AUS_FOLDS)

In [ ]:
grid_name = 'bert-{0}-fold-{1}-combine-max.run'
cros_val_name = 'bert'
#grid_name = 'dpbtsoph-{0}-fold-{1}-combine-max.run'

runs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', [os.path.join(BASE_DIR, 'grid-search', grid_name.format(y, x)) for x in range(0, 5) for y in ['RR', 'ERR', 'R@20', 'NDCG', 'RBP']], per_query=True)

In [ ]:
names = [os.path.join(grid_dir, grid_name.format(y, x)) for x in range(0, 5) for y in ['RR', 'ERR', 'R@20', 'NDCG', 'RBP']]

In [ ]:
# [names[x:x+5] for x in range(0, len(names), 5)]

## results for test dataset for runs

In [ ]:
test_res = []
for r in range(5):
    test_res.append([])
run_folds = [runs[x:x+5] for x in range(0, len(runs), 5)]
for filters, f in zip(tt_folds, run_folds):
    for i, measure in enumerate(f):
        test_res[i].append(measure[measure.index.isin(filters[1])].mean())

In [ ]:
metrics = copy.copy(config.METRIC_NAMES)
del metrics['unjudged@20']
del metrics['recall_100']

test_res_avg = pd.DataFrame()
for m, r in zip(metrics, test_res):
    test_res_avg[m] = pd.DataFrame(r)[m]
    print('-'*20)
    
test_res_avg = test_res_avg.rename(metrics, axis='columns')

In [ ]:
fig, axs = plt.subplots(1, 5)
fig.set_size_inches(16, 10)
for i, m in enumerate(RERANK_METRICS.items()):
    print(m)
    p = test_res_avg[m[1]].plot.box(ax=axs[i], fontsize=15, boxprops=dict(linestyle='-', linewidth=2), medianprops=dict(linestyle='-', linewidth=2), color=dict(boxes='black', whiskers='black', medians='b', caps='r'), figsize=(16, 4))
    p.axhline(y=base_df[m[0]], xmin=0.0, xmax=1.0, linestyle='--', linewidth=1.0, color='blue')
    p.axhline(y=sdm_base_df[m[0]], xmin=0.0, xmax=1.0, linestyle='--', linewidth=1.0, color='green')
    axs[i].set_ylim(top=RERANK_YLIMS[i])

fig.tight_layout()
# fig.savefig('figures/ausnl-{0}-cross-val.pdf'.format(cros_val_name))

In [ ]:
dfs = []
model_names = ['BERT', '4ft', '5ft']
metric_names = ['RR', 'ERR', 'R@20', 'NDCG', 'RBP']
metric_ll = ['recip_rank', 'err@20', 'recall_20', 'ndcg', 'rbp@0.80']
for m in metric_names:
    dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', 
                        [os.path.join(BASE_DIR, 'grid-search', '{0}-{1}-combine-max.run'.format(r, m)) for r in ['bert', '4feat', '5feat']], per_query=True))

In [ ]:
print(len(dfs[0]))
dfs[0][2].mean()[RERANK_METRICS]

In [ ]:
out_df = pd.DataFrame(columns=metric_names)
for d, m, l in zip(dfs, metric_names, metric_ll):
    out_df[m] = [x[l].mean() for x in d]
    
    print([stats.ttest_rel(d[0][l], x[l]).pvalue for x in d[1:]])
    
out_df.index = model_names
print(out_df.round(4).to_latex())

# write_table('tables/ausnl-multi', bold_max(out_df.round(4)).rename(columns=metrics).to_latex(escape=False))

In [ ]:
tmp_df = pd.DataFrame(columns=RERANK_METRICS)
for n, d in zip(model_names, dfs):
    cross = cross_validation(d, tt_folds, RERANK_METRICS, base_qry)
    tmp_df.loc[n] = cross[0]
    
# write_table('tables/ausnl-multi', bold_max(tmp_df).rename(columns=metrics).to_latex(escape=False))

In [ ]:
dfs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', [os.path.join(BASE_DIR, 'grid-search', '{0}-RBP-combine-max.run'.format(r)) for r in ['bert', '4feat', '5feat']], per_query=True)

In [ ]:
def read_grid_res(path: str, folds: int):
    with open(path) as f:
        measures = f.readline().split()
        fold_res = [[] for x in measures]
        weights = [[] for x in measures]
        # read the fold score
        for i in range(folds):
            parts = f.readline().split()
            for j in range(len(parts)):
                fold_res[j].append(float(parts[j]))
            
        #read the weights
        for i in range(folds):
            parts = f.readline().split('] [')
            parts = [x.strip(' []\n') for x in parts]
            for j in range(len(parts)):
                weights[j].append([float(x) for x in parts[j].split()])
                
        return measures, fold_res, weights
             
_4res = read_grid_res(os.path.join(BASE_DIR, 'grid-search', '4feat-results.txt'), 5)

In [ ]:
_5res = read_grid_res(os.path.join(BASE_DIR, 'grid-search', '5feat-results.txt'), 5)

In [ ]:
bertres = read_grid_res(os.path.join(BASE_DIR, 'grid-search', 'bert-results.txt'), 5)

In [ ]:
rev_metrics = {v:k for k,v in RERANK_METRICS.items()}
rev_metrics['ERR'] = rev_metrics['ERR@20']

In [ ]:
fig, axs = plt.subplots(1, 5)
fig.set_size_inches(16, 10)
cnt = 0
for i in [2, 3, 4, 1, 0]:
    a = pd.DataFrame()
    a['x'] = [0 for x in range(5)]
    a['x1'] = [1 for x in range(5)]
    a['x2'] = [2 for x in range(5)]
    a['bert'] = bertres[1][i]
    a['4ft'] = _4res[1][i]
    a['5ft'] = _5res[1][i]
    p = a.plot.scatter(y='bert', x='x', ax=axs[cnt], fontsize=15, figsize=(16, 4))
    a.plot.scatter(y='4ft', x='x1', ax=axs[cnt], fontsize=15, figsize=(16, 4))
    a.plot.scatter(y='5ft', x='x2', ax=axs[cnt], fontsize=15, figsize=(16, 4))
    p.axhline(y=base_df[rev_metrics[_4res[0][i]]], xmin=0.0, xmax=1.0, linestyle='--', linewidth=1.0, color='b')
    p.set(xlabel=None)
    p.set(xticks=[0, 1, 2])
    p.set_ylabel(_4res[0][i], fontsize=15)
    p.set_xticklabels(['bert', '4ft', '5ft'], rotation='vertical', fontsize=18)
    cnt += 1

fig.tight_layout()
# fig.savefig('figures/ausnl-comb-train-res.pdf')

In [ ]:
fig, axs = plt.subplots(1, 5)
fig.set_size_inches(16, 10)
for i, m in enumerate(_4res[0]):
    a = pd.DataFrame()
#     a['x'] = [0 for x in range(5)]
# #     a['x1'] = [1 for x in range(5)]
# #     a['x2'] = [2 for x in range(5)]
# #     a['bert'] = bertres[2][i]
# #     a['4ft'] = _4res[2][i]
#     a['5ft'] = _5res[2][i]
#     print(a)
#     break
#     p = a.plot.scatter(y='bert', x='x', ax=axs[i], fontsize=15, figsize=(16, 4))
#     a.plot.scatter(y='4ft', x='x1', ax=axs[i], fontsize=15, figsize=(16, 4))
#     a.plot.scatter(y='5ft', x='x2', ax=axs[i], fontsize=15, figsize=(16, 4))
#     p.axhline(y=base_df[rev_metrics[m]], xmin=0.0, xmax=1.0, linestyle='--', linewidth=1.0, color='b')
#     p.set(xlabel=None)
#     p.set(xticks=[0, 1, 2])
#     p.set_ylabel(m, fontsize=15)
#     p.set_xticklabels(['bert', '4ft', '5ft'], rotation='vertical', fontsize=18)

In [ ]:
all_weight_df = pd.DataFrame()

for i in range(5):
    tmp = pd.DataFrame(_4res[2][i])
    tmp['measure'] = _4res[0][i]
    tmp['x'] = 0
    all_weight_df = all_weight_df.append(tmp)
    
# all_weight_df.plot.scatter(x='measure', y=0)

In [ ]:
tmp = pd.DataFrame()

In [ ]:
tmp['0cnt'] = all_weight_df.groupby('measure')[0].value_counts()
tmp['1cnt'] = all_weight_df.groupby('measure')[1].value_counts()

In [ ]:
tmp

In [ ]:
def create(res, feature_names, folds=5):
    xs = [[x]*5 for x in range(len(feature_names))]
    
    df = pd.DataFrame()

    for i in range(5):
        tmp = pd.DataFrame(res[2][i])
        tmp['measure'] = res[0][i]
        tmp['x'] = 0
        df = df.append(tmp)

    df2 = pd.DataFrame()
    for i in range(len(feature_names)):
        df2[i] = df.groupby('measure')[i].value_counts()

    vals = {}

    for iter in df2.iterrows():
        v = vals.get(iter[0][0], [])
        start = len(v) == 0
        for i in range(int(iter[1][0])):
            v.append(iter[0][1])
        # v.append((iter[0][1], ))
        if start:
            vals[iter[0][0]] = v
    
    df2 = pd.DataFrame(vals)
    
    fig, axs = plt.subplots(1, 5)
    fig.set_size_inches(16, 6)
    inds = [2, 3, 4, 1, 0]
    for i, ind in enumerate(inds):
        # for j in range(len(feature_names)):
            # legend = False
        sns.scatterplot(xs[0], df2[res[0][i]], alpha=0.8, ax=axs[i], markers=['X'], style=0, legend=False, s=40)
        axs[i].set_xticks([0])
        axs[i].set_ylabel(res[0][ind], size=20)
        axs[i].set_ylim(0, 1)
        axs[i].set_xticks([])
        fig.legend(feature_names, frameon=True, ncol=1, bbox_to_anchor=[0.99, 0.97], prop={"size": 12}, markerfirst=False).get_frame().set_edgecolor('black')
    
    fig.tight_layout()
    
    return fig
    
fig = create(_4res, ['DP', 'BERT', 'court', 'oprob'])
# fig.savefig('figures/ausnl-4ft-weights.pdf')

In [ ]:
fig = create(_5res, ['DP', 'BERT', 'court', 'oprob', 'fact'])
# fig.savefig('figures/ausnl-5ft-weights.pdf')

In [ ]:
fig = create(bertres, ['DP', 'BERT'])
# fig.savefig('figures/ausnl-bert-weights.pdf')